In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('WindowStreaming').getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from time import sleep

In [ ]:
userSchema = StructType().add("Acc No", "integer").add("Transaction", "string").add("Time Stamp", "string").add("Amount", "integer")
streaming = spark.readStream.schema(userSchema) \
    .option('maxFilesPerTrigger', 1)\
    .json('/content/drive/MyDrive/BDA_1_Sem/ABD/ABD_Lab/Hadoop/Stream/data/')

accountDF = streaming.withColumn('Times', to_timestamp(col('Time Stamp'))).drop('Time Stamp')

spark.conf.set("spark.sql.shuffle.partitions", 1)

In [ ]:
accCount = accountDF.groupBy('Acc No').count()
accountQuery = accCount.writeStream.queryName('count_accounts1')\
    .format('memory')\
    .outputMode('complete')\
    .start()

for x in range(10):
  spark.sql('SELECT * FROM count_accounts').show()
  sleep(5)

#accountQuery.awaitTermination()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `count_accounts` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [count_accounts], [], false


In [ ]:
lessamount = accountDF.select('Acc No', 'Amount').where(col('Amount') < 2000).groupBy('Acc No').count()
lessquery = lessamount.writeStream.queryName('Min_Amount')\
    .trigger(processingTime ="20 seconds")\
    .format('console')\
    .outputMode('complete')\
    .start()
lessquery.awaitTermination()



lessamount = accountDF.select('Acc No', 'Amount').where(col('Amount') < 2000)
lessquery = lessamount.writeStream.queryName('Min_Amount')\
    .trigger(processingTime ="10 seconds")\
    .format('console')\
    .outputMode('append')\
    .start()
lessquery.awaitTermination()



windowedCount = accountDF \
    .withWatermark("Times", "10 minutes") \
    .groupBy(col('Acc No'),"Times") \
    .count()

query = windowedCount.writeStream \
    .format("csv") \
    .trigger(processingTime="10 seconds") \
    .option("checkpointLocation", "/content/drive/MyDrive/BDA_1_Sem/ABD/ABD_Lab/Hadoop/Stream/Streaming/checkpoint") \
    .option("path", "/content/drive/MyDrive/BDA_1_Sem/ABD/ABD_Lab/Hadoop/Stream/Streaming/outputDir") \
    .outputMode("append") \
    .start() \
    .awaitTermination()


windowedCount = accountDF.select('Acc No', 'Amount', 'Transaction', 'Times').where(col('Amount') > 70000) \
    .withWatermark("Times", "10 minutes")

query = windowedCount.writeStream \
    .format("csv") \
    .trigger(processingTime="10 seconds") \
    .option("checkpointLocation", "/content/drive/MyDrive/BDA_1_Sem/ABD/ABD_Lab/Hadoop/Stream/Streaming/checkpoint") \
    .option("path", "/content/drive/MyDrive/BDA_1_Sem/ABD/ABD_Lab/Hadoop/Stream/Streaming/outputDir/") \
    .outputMode("append") \
    .start() \
    .awaitTermination()
